In [ ]:
import os
import matplotlib.pyplot as plt
import h5py
import numpy as np
import networkx as nx
import yaml

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 12,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
results_dir_path = f"{script_dir}/outputs/qbp/very_large/2024-04-30/18-07-06"
# ---------------------------------------------------------------------------
results_path = f"{results_dir_path}/result"
config_path = f"{results_dir_path}/.hydra/config.yaml"
with open(config_path, "r") as f:
    raw_config = f.read()
    config = yaml.safe_load(raw_config)
layers_number = int(config["quantum_annealing_schedule"]["steps_number"])
with h5py.File(results_path, "r") as f:
    circuit = nx.Graph()
    fields = np.array(f["fields"])
    pairs = np.array(f["coupled_spin_pairs"])
    tn_graph = nx.Graph()
    for id in range(fields.shape[0]):
        tn_graph.add_node(id)
    front_layer_ids = { key: value for key, value in zip(range(fields.shape[0]), range(fields.shape[0])) }
    for layer in range(layers_number):
        for pair in pairs:
            id = (layer, pair[0], pair[1])
            tn_graph.add_node(id)
            tn_graph.add_edge(id, front_layer_ids[pair[0]])
            tn_graph.add_edge(id, front_layer_ids[pair[1]])
            front_layer_ids[pair[0]] = id
            front_layer_ids[pair[1]] = id
        for i in range(fields.shape[0]):
            id = (layer, i)
            tn_graph.add_node(id)
            tn_graph.add_edge(id, front_layer_ids[i])
            front_layer_ids[i] = id
    fig = plt.figure(figsize=(20, 20))
    nx.draw_random(tn_graph, node_size=3, width=0.5, node_color='red')

In [ ]:
fig.savefig(f"{results_dir_path}/tensor_network.pdf", bbox_inches="tight")